# Práctico: Ensembles en Machine Learning

En este práctico exploraremos diferentes técnicas de ensamblado de modelos (ensembles) usando `scikit-learn`. Los ensembles permiten combinar varios modelos base para mejorar el rendimiento y la robustez de las predicciones.

## Objetivos

- Comprender el funcionamiento de los métodos de ensemble más populares: Bagging, Random Forest, AdaBoost y Gradient Boosting.
- Comparar el desempeño de un árbol de decisión individual con modelos ensemble.
- Analizar la importancia de las variables y la estabilidad de los modelos.

---

## 1. Carga de datos

Utilizaremos el dataset de vinos (`wine`) incluido en `scikit-learn`. Este dataset es útil para clasificación multiclase y tiene variables numéricas.

In [ ]:
from sklearn.datasets import load_wine
import pandas as pd

data = load_wine()
X = data.data
y = data.target
df = pd.DataFrame(X, columns=data.feature_names)
df['target'] = y
df.head()

In [ ]:
# Imprimir la descripción del dataset
print(data.DESCR)

## 2. Árbol de Decisión Individual

Entrenaremos un árbol de decisión y evaluaremos su desempeño como modelo base.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tree = DecisionTreeClassifier(max_depth=3, random_state=42)
tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_test)

print(classification_report(y_test, y_pred_tree))

In [ ]:
# Visualización del árbol entrenado
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
plot_tree(tree, feature_names=data.feature_names, class_names=data.target_names, filled=True)
plt.title("Árbol de Decisión Individual (max_depth=3)")
plt.show()

## 3. Bagging

El Bagging (Bootstrap Aggregating) entrena varios modelos independientes sobre subconjuntos aleatorios de los datos y promedia sus predicciones.

In [ ]:
from sklearn.ensemble import BaggingClassifier

bagging = BaggingClassifier(DecisionTreeClassifier(), n_estimators=100, random_state=42)
bagging.fit(X_train, y_train)
y_pred_bag = bagging.predict(X_test)

print(classification_report(y_test, y_pred_bag))

In [ ]:
# Visualización de uno de los árboles del ensemble de Bagging
plt.figure(figsize=(12, 8))
plot_tree(bagging.estimators_[0], feature_names=data.feature_names, class_names=data.target_names, filled=True)
plt.title("Ejemplo de Árbol en Bagging")
plt.show()

In [ ]:
# Curva de error en train y test variando n_estimators en Bagging
import numpy as np
from sklearn.metrics import accuracy_score

n_range = range(1, 101, 5)
train_scores = []
test_scores = []

for n in n_range:
    bag = BaggingClassifier(DecisionTreeClassifier(), n_estimators=n, random_state=42)
    bag.fit(X_train, y_train)
    train_scores.append(1 - accuracy_score(y_train, bag.predict(X_train)))
    test_scores.append(1 - accuracy_score(y_test, bag.predict(X_test)))

plt.figure(figsize=(8, 5))
plt.plot(n_range, train_scores, label="Error Train")
plt.plot(n_range, test_scores, label="Error Test")
plt.xlabel("n_estimators")
plt.ylabel("Error")
plt.title("Error vs n_estimators en Bagging")
plt.legend()
plt.show()

**Ejercicio:** Implementa un BaggingClassifier usando regresión logística como estimador base (`LogisticRegression`). Compara su desempeño con el Bagging de árboles y discute en qué casos podría ser preferible cada uno.

## 4. Random Forest

Random Forest es una extensión de Bagging donde cada árbol ve solo un subconjunto aleatorio de variables en cada división.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print(classification_report(y_test, y_pred_rf))

In [ ]:
# Visualización de uno de los árboles del Random Forest
plt.figure(figsize=(12, 8))
plot_tree(rf.estimators_[0], feature_names=data.feature_names, class_names=data.target_names, filled=True)
plt.title("Ejemplo de Árbol en Random Forest")
plt.show()

**Ejemplo:** Visualiza la importancia de las variables en el Random Forest.

In [ ]:
importances = rf.feature_importances_
plt.figure(figsize=(10, 6))
plt.barh(data.feature_names, importances)
plt.xlabel("Importancia")
plt.title("Importancia de las variables según Random Forest")
plt.show()

In [ ]:
# Curva de error en train y test variando n_estimators en Random Forest
train_scores_rf = []
test_scores_rf = []

for n in n_range:
    rf_tmp = RandomForestClassifier(n_estimators=n, random_state=42)
    rf_tmp.fit(X_train, y_train)
    train_scores_rf.append(1 - accuracy_score(y_train, rf_tmp.predict(X_train)))
    test_scores_rf.append(1 - accuracy_score(y_test, rf_tmp.predict(X_test)))

plt.figure(figsize=(8, 5))
plt.plot(n_range, train_scores_rf, label="Error Train")
plt.plot(n_range, test_scores_rf, label="Error Test")
plt.xlabel("n_estimators")
plt.ylabel("Error")
plt.title("Error vs n_estimators en Random Forest")
plt.legend()
plt.show()

## 5. AdaBoost

AdaBoost ajusta secuencialmente modelos, enfocándose en los ejemplos mal clasificados por los modelos anteriores.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression

ada = AdaBoostClassifier(estimator=LogisticRegression(max_iter=10000), n_estimators=50, learning_rate=1.0, random_state=42)
ada.fit(X_train, y_train)
y_pred_ada = ada.predict(X_test)

print(classification_report(y_test, y_pred_ada))

In [ ]:
# Curva de error en train y test variando n_estimators en AdaBoost
train_scores_ada = []
test_scores_ada = []

for n in n_range:
    ada_tmp = AdaBoostClassifier(n_estimators=n, learning_rate=1.0, random_state=42, algorithm="SAMME")
    ada_tmp.fit(X_train, y_train)
    train_scores_ada.append(1 - accuracy_score(y_train, ada_tmp.predict(X_train)))
    test_scores_ada.append(1 - accuracy_score(y_test, ada_tmp.predict(X_test)))

plt.figure(figsize=(8, 5))
plt.plot(n_range, train_scores_ada, label="Error Train")
plt.plot(n_range, test_scores_ada, label="Error Test")
plt.xlabel("n_estimators")
plt.ylabel("Error")
plt.title("Error vs n_estimators en AdaBoost")
plt.legend()
plt.show()

**Ejercicio:** Implementa un modelo de Boosting utilizando regresión logística como estimador base (`LogisticRegression`). Compara su desempeño con el Boosting basado en árboles y discute en qué situaciones podría ser útil esta alternativa.

## 6. Gradient Boosting

Gradient Boosting construye modelos secuenciales, cada uno corrigiendo los errores del anterior usando gradientes.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
gb.fit(X_train, y_train)
y_pred_gb = gb.predict(X_test)

print(classification_report(y_test, y_pred_gb))

In [ ]:
# Curva de error en train y test variando n_estimators en Gradient Boosting
train_scores_gb = []
test_scores_gb = []

for n in n_range:
    gb_tmp = GradientBoostingClassifier(n_estimators=n, learning_rate=0.1, random_state=42)
    gb_tmp.fit(X_train, y_train)
    train_scores_gb.append(1 - accuracy_score(y_train, gb_tmp.predict(X_train)))
    test_scores_gb.append(1 - accuracy_score(y_test, gb_tmp.predict(X_test)))

plt.figure(figsize=(8, 5))
plt.plot(n_range, train_scores_gb, label="Error Train")
plt.plot(n_range, test_scores_gb, label="Error Test")
plt.xlabel("n_estimators")
plt.ylabel("Error")
plt.title("Error vs n_estimators en Gradient Boosting")
plt.legend()
plt.show()

## 7. Comparación de Resultados

Resume y compara los resultados obtenidos con cada técnica. ¿Cuál fue el mejor modelo? ¿Por qué?

## 8. Preguntas para reflexionar

- ¿Por qué los ensembles suelen superar a los modelos individuales?
- ¿Qué riesgos tiene usar ensembles muy complejos?
- ¿Cómo elegir entre Bagging y Boosting?